# Introduction to PyCBC Inference 2: Analyzing a gravitational wave
### Collin Capano and Alex Nitz

In this tutorial we show how to set up a run for a gravitation wave, in this case, GW150914.

### Prerequisites

We will need the most recent version of pycbc installed for this tutorial.

In [ ]:
import sys
!{sys.executable} -m pip install pycbc lalsuite ligo-common --no-cache-dir

# This is needed to access the executables on sciserver. On a personal machine this should be ignore.
path = %env PATH
%env PATH=$path:/home/idies/miniconda3/envs/py37/bin 

In [ ]:
import os
from IPython.display import Image
from IPython.core.display import HTML

## BBH example

To analyze a gravitational wave, we need to provide a prior, a model, and data settings in our configuration file. These settings can all be put into a single file, or split up into multiple files, all of which are provided via the `--config-file` option. In this example, we split the configuration up into 3 files:
  * [gw150914_like.ini](gw150914_like.ini): specifies the model we will use, and the parameters that will be varied and their priors. This is the configuration file you would use to analyze signals like GW150914.
  * [emcee_pt-gw150914_like.ini](emcee_pt-gw150914_like.ini): specifies sampler settings. In this case, we are using the `emcee_pt` sampler.
  * [data.ini](data.ini): specifies the settings for loading the gravitational-wave data.

In the following sections, we look at the various sections of thes configuration files in detail.

## The model

The model we will use is set by the `[model]` section. By setting the `name = gaussian_noise`, we tell `pycbc_inference` that we want to use the [GaussianNoise](https://pycbc.org/pycbc/latest/html/pycbc.inference.models.html#pycbc.inference.models.gaussian_noise.GaussianNoise) model. This means that we will be analyzing data $d$ from one or more detectors, which is sampled at some constant sample rate $1/\Delta t$ for a period of time $T$, yielding $N = T/\Delta t$ samples (more on how to load data below).

This model assumes that the data consists of stationary Gaussian noise plus a signal $h$. The signal is modeled by a waveform model, which depends on several parameters. Which model to use is determined by the `approximant` parameter, which is set in the `static_params` section (see below).

When this model is given a set of parameter values $\vec{\vartheta}$, it generates a frequency-domain waveform $\tilde{h}(\vec{\vartheta})$ using the waveform model. It then calculates the log likelihood:
\begin{equation}
\log p(d|\vec{\vartheta}, h) =  -\frac{1}{2} \sum_i \left< h_i(\vec{\vartheta}) - d_i,\, h_i(\vec{\vartheta}) - d_i \right>,
\end{equation}
where the sum is over the number of detectors. The inner product is given by:
\begin{equation}
\left<a_i, \, b_i\right> = 4 \Re \sum_{k=k_0}^{N/2} \frac{\tilde{a}_i^{*}(k \Delta f) \tilde{b}_i(k \Delta f)}{S^{(i)}_n(k\Delta f)} \Delta f.
\end{equation}
Here, $S_n^{(i)}$ is the power spectral density of the noise in the $i$th detector and $\Delta f = 1/T$ is the frequency resolution. This is the discrete form of the matched filter.

As we see, the model requires a lower frequency cutoff for the inner product $f_0 = k_0 \Delta f$. This is set with the `low-frequency-cutoff` option, which we have set to 20.

*Tip:* By setting the `low-frequency-cutoff` to a single value, we will use the same cutoff for all detectors. If you would like to use a different cutoff for each detector, set `low-frequency-cutoff = {DET1}:{F1} {DET2}:{F2} [...]`, where `{DETn}` is the name of the `n`-th detector (e.g., `H1`) and `{Fn}` is the frequency to use for detector `n`.

## The variable and static params

As with the Normal 2D example, we see that there is a `[variable_params]` section. This lists the names of all of the parameters we will be varying in the run. Now we see that there is also a `[static_params]` section. These are parameters that will be kept fixed throughout the run. In this example we have:
 * `approximant = IMRPhenomPv2`: this means that the waveform model we use will be IMRPhenomPv2
 * `f_lower = 20`: this sets the starting frequency for the waveform generation to 20Hz. *This is separate from the low frequency cutoff of the inner product, which is set in the `[model]` section. Generally, this number should be less-than-or-equal-to the `low-frequency-cutoff` in the `[model]` section.*
 * `f_ref = 20`: the "reference" frequency of the waveform.

The `approximant` argument determines what type of waveform we will be generating. Since it is set to `IMRPhenomPv2`, we will be generating a CBC waveform.

## Setting the prior

**Every parameter listed in the `[variable_params]` section must have a prior specified in the file.** This is done by adding sections named `[prior-{params}]` to the file, where `{param}` is the name of the parameter that the section sets a prior for; e.g., `[prior-mass1]`. A section may provide a distribution for multiple parameters. In that case, all of the parameters must be listed in the header as a `+` separated list. For example, `[prior-ra+dec]`. The order that the parameters are provided does not matter.

Each prior section must also have a `name` set. This specifies the name of the distribution to use for that parameter. Distributions are defined in PyCBC's [distributions package](https://pycbc.org/pycbc/latest/html/pycbc.distributions.html); several are available for use. For the complete list, see the table [here](https://pycbc.org/pycbc/latest/html/inference.html#configuring-the-prior).

The rest of the settings in the `[prior]` sections depend on the distribution being used. For example, the `uniform` distribution requires minimum and maximum bounds to be provided for each parameter:
```
[prior-mass1]
name = uniform
min-mass1 = 10.
max-mass1 = 80.
```
Some distributions require no options, since they are predefined in the code. For example, the `uniform_sky` distribution provides the appropriate distributions for right ascension and declination, which it expects to be called `ra` and `dec`, respectively. This is why that section is simply:
```
[prior-ra+dec]
name = uniform_sky
```

### Challenge:
 * The configuration file is missing a prior for the mass of the second object, `mass2`. Set a prior on `mass2` that is uniform in $[10,80)\,\mathrm{M}_\odot$. The solution can be found [here](http://pycbc.org/pycbc/latest/html/inference/examples/bbh.html#setup-the-configuration-files).

## The sampler

The sampler settings are in [emcee_pt-gw150914_like.ini](emcee_pt-gw150914_like.ini). Here we are using the `emcee_pt` sampler. This is a parallel tempered sampler, so it requires a number of temperatures to be set. Note a few other differences from the analytic example here:
  * The `burn-in-test` is set to `nacl & max_posterior`.
  * Instead of an `niterations` option, we have `effective-nsamples = 1000`.
  * A checkpoint interval is set: `checkpoint-interval = 2000`
  * There is a `max-samples-per-chain` option.

These are all options specific to MCMC samplers like `emcee_pt`. Details:

#### The burn in option
Multiple burn in tests may be combined using standard boolean operators like `&` and `|`. In this example, we will consider the sampler to be burned in when it has passed the `nacl` test *and* the `max_posterior` test. This means:

 * `nacl`: The second half of the chain must be longer than 5 times the ACL. If so, the samlper is considered burned-in at the halfway point.
 * `max_posterior`: All of the walkers must find a point that has a log posterior greater than `maxP - ndim/2`, where `maxP` is the maximum posterior value found over all the walkers and `ndim` is the number of variable parameters. The first iteration for which all the walkers pass this test is the burn in iteration.
 
By doing `&`, we take the larger iteration of these two tests. This combination of tests has worked well for `emcee_pt`.

#### Checkpointing
When a `checkpoint-interval` is set, `pycbc_inference` will dump the results to a checkpoint file after every `checkpoint-interval` iterations. The checkpoint file has the same name as the output, but with `.checkpoint` added on to it. 

While ``pycbc_inference`` is running it will create a checkpoint file which
is named ``{output-file}.checkpoint``, where ``{output-file}`` was the name
of the file you specified with the ``--output-file`` command. If a `checkpoint-interval` is set, `pycbc_inference` will checkpoint after the given number of iterations. For `emcee_pt`, this means that it will dump the current samples to this file; when finished, the file is
renamed to ``{output-file}``.

A ``{output-file}.bkup`` is also created, which is a copy of the checkpoint file. This is kept in case the checkpoint file gets corrupted during writing. The ``.bkup`` file is deleted at the end of the run, unless ``--save-backup`` is turned on.

If `pycbc_inference` is terminated while running (either by error, or by a system interrupt), the checkpoint and bkup files remain. When `pycbc_inference` is restarted, it will check for those files. If they are found, it will resume from where it last left off.

#### Termination condition
By setting `effective-nsamples` we tell the `pycbc_inference` until it has an effective number of samples greater than or equal to the specified value. Effective samples are the number of independent samples of the posterior we have. This is given by number of samples that remain after burn-in and thinned by the autocorelation time.

The number of effective samples are counted at each checkpoint. For this reason, a checkpoint-interval must be provided if `effective-nsamples` is set.

#### Max samples per chain
If `max-samples-per-chain` is provided, `pycbc_inference` will ensure that no more than the given number of samples per chain are stored in the output file. Samples will be thinned on disk and in memory when a checkpoint happens to ensure this. This is important for keeping file size down. Without it, a GW run with `200` walkers and `20` temps can result in a file that is over 100GB, since every sample will be saved. With `max-samples-per-chain = 1000`, the maximum file size is capped to ~1GB.

### Samping and waveform transforms
You'll note a `waveform_transforms` and `sampling_transforms` sections. Those are described in more detail in the next tutorial.

## Data settings

The settings for loading the data are in [data.ini](data.ini). This contains a `[data]` section, which is read by the GaussianNoise model to figure out what data to load, and how to condition. Here is what each of the settings that we have does:
 * `instruments`: This tells the code what detectors to analyze. Here, we've set it to `H1` and `L1`.
 * `trigger-time`, `analysis-start-time` and `analysis-end-time`: The `analysis-(start|end)-time` options determine the time that will be analyzed. Notice that the start-time is `-8` and the end time is `2`. This is because these times are measured with respect to the `trigger-time` option. Here, we put an estimate of the GPS time when the binary black hole merger occurred (in a Geocentric reference frame). With these settings, our analyzed time will start 8 seconds before the merger time and end 2 seconds after.
 * `psd-estimation`: This determines how we will estimate the PSD. By setting it to `median-mean`, the PSD will be analyzed from the data using a Welch-like method. Basically, the data is chopped up into semi-overlapping segments, an FFT is taken in each block, then the median is taken over all odd-numbered segments. The same process is repeated for the even-numbered segments. The two sets are then averaged to give the PSD.
 * `psd-start-time` and `psd-end-time`: This defines the analysis block that is used for estimating the PSD. To get a good estimate, you generally want to use ~512s of data. Here, we use 512s centered on the trigger time.
 * `psd-segment-length` and `psd-segment-stride`: These determine the size of each segment when doing the median-mean method, and how much each segment overlaps.
 * `psd-inverse-length`: This truncates the inverse PSD in the time domain to be 4s long.
 * `frame-files`: Specifies the gravitational-wave data ("frame") files to use.
 * `channel-name`: The name of the channels in the frame files containing the gravitational-wave data to analyze.
 * `sample-rate`: The sample rate we will use for the analysis. You want this to be atleast twice the maximum frequency of any possible waveform that will be generated by your prior. For BBH, 2048Hz is generally ok.
 * `strain-high-pass`: Causes a high-pass filter to be applied to the data when it is first loaded, with the cutoff frequency (here) set to 15Hz. This is just to remove the large amplitude low-frequency noise, so as not to cause numerical overflow issues when calculating the likelihood. Generally, you want this to be a few Hz lower than the low-frequency-cutoff used in the model.
 * `pad-data`: Adds an extra few seconds on to the data when loading. This is to avoid corruption issues from the `strain-high-pass` filter. The padded data are removed after the high-pass filter is applied, and before any FFTs are done.

### Determining analysis time duration

Why did we use -8 and +2 for the analysis times? The discrete inner product treats the data as if it were cyclic. If we try to filter a model waveform that is longer (starting from the low-frequency-cutoff) than the analysis duration, it will wrap around to the beginning. For example, if we used an analysis time of 4s, but a waveform is 5s long, the last second of the waveform will wrap around to lay on top of the first second of the segment. To avoid this, we need to analyze a segment that is longer than the longest possible waveform admitted by our prior plus our uncertainty in the trigger time ($\pm 0.1\,$s). In this case, our longest waveform is $m_1 = m_2 = 10\,\mathrm{M}_\odot$. We can check the duration of this waveform using `get_waveform_filter_length_in_time`:

In [ ]:
from pycbc import waveform

In [ ]:
waveform.get_waveform_filter_length_in_time(approximant='IMRPhenomPv2', mass1=10., mass2=10.,
                                            spin1z=0.99, spin2z=0.99, f_lower=20.)

Since the `trigger-time` is near the merger time, using ~8s before and ~2s after sufficiently encompasses the longest waveform we might sample.

### Challenge:

What analysis-time settings would you use if your prior on mass1 and mass2 was uniform in $[5, 40)\,\mathrm{M}_\odot$, and you were starting from 20Hz? What if you start from 15Hz?

## Run it

Before we can run, we need to download frame files from [GWOSC](https://www.gw-openscience.org/about/). These contain the LIGO data that we will analyze.

In [ ]:
if not os.path.exists('H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf'):
    !wget https://www.gw-openscience.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf
if not os.path.exists('L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf'):
    !wget https://www.gw-openscience.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf

To run this example, we will just run the bash script. Running this example to completion will take several hours. Instead, we'll just run for a couple of checkpoints, kill it, then start it again to see how checkpointing works.

First, do the following:
 * Set the checkpoint interval in the config file to 4.

Now run the script. Watch the output. After it checkpoints (it will say "Running sampler for 4 to 8 iterations"), interrupt the kernel by hitting the stop button above.

In [ ]:
!bash run.sh

Now run `ls` to see the files in the diretory. You should see a `bbh_results.hdf.checkpoint` and `bbh_results.hdf.bkup`. These are your checkpoint and backup files:

In [ ]:
!ls

Run the script again. Watch the messages. You should see it say that it is starting from iteration 4. Stop it after it has gotten to another checkpoint.

In [ ]:
!bash run.sh

We can replace `effective-nsamples` by `niterations` in the sampler config file to get this to complete now. Remove or comment out the `effective-nsamples` option, and replace it with: `niterations = 12`. Now re-run. You should see `pycbc_inference` start up from the last checkpoint, but stop as soon as it gets to 12 iterations.

In [ ]:
!bash run.sh

Note that the checkpoint file has been renamed to `bbh_results.hdf`, and the backup file is deleted.

In [ ]:
!ls

Since we ran for such a short period of time, the samples in `bbh_results.hdf` will look nothing like the posterior. In the next tutorial, we will take a look at a completed result file which has been run for the full time.